In [75]:
# for scrap_steam250
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output

# for data preprocessing & analysis
import pandas as pd
from datetime import datetime as dt

# for scrap_steamspy
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
steam_spy = 'https://steamspy.com/tag/viking'

In [3]:
request = requests.get(steam_spy)
soup = BeautifulSoup(request.content, "html.parser")

In [25]:
games = soup.find_all("tr")

In [181]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get(steam_spy)

C:\Users\Nastya\AppData\Local\Temp\ipykernel_16088\2634746696.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [196]:
def scrap_steamspy(driver):
    links = []
    owners = []
    release_dates = []
    for i in range(4):
        links_on_page = driver.find_elements(By.XPATH, "//a[starts-with (@href, '/app/')]")
        owners_on_page = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'sorting_1')]
        release_dates_on_page = [elem.get_attribute('data-order') for elem in driver.find_elements(By.CLASS_NAME, 'treleasedate') if elem.get_attribute('data-order')]
        
        links.extend([link.get_attribute('href') for link in links_on_page])
        owners.extend(owners_on_page)
        release_dates.extend(release_dates_on_page)
        
        next_page_button = driver.find_element(By.CLASS_NAME, "next")
        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        next_page_button.click()
    return links, owners, release_dates

In [197]:
links, owners, release_dates = scrap_steamspy(driver)

In [205]:
def get_info_from_steam(steam_link):
    response = requests.get(steam_link)
    soup = BeautifulSoup(response.content, "html.parser")
    reviews_count = soup.find('meta', {'itemprop': 'reviewCount'}).get('content')
    positive_reviews = soup.find('div', {'itemprop': 'aggregateRating'}).get('data-tooltip-html').split('%')[0]
    return reviews_count, positive_reviews

In [255]:
err_links = []

In [256]:
def get_pc_games_info(links):
    imgs = []
    steam_links = []
    reviews_counts = []
    positive_reviews = []
    games_names = []
    for link in links:
        try:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, "html.parser")
            img = soup.find('img', {'class': 'img-responsive'}).get('src')
            steam_link = soup.find(lambda tag: tag.name == "a" and 'Store' in tag.text).get('href')
            game_name = soup.find('h3').text
            reviews_count, positive_review = get_info_from_steam(steam_link)
        except:
            err_links.append(link)
        
        games_names.append(game_name)
        imgs.append(img)
        steam_links.append(steam_link)
        reviews_counts.append(reviews_count)
        positive_reviews.append(positive_review)
        
        print('Processed %.2f%% of data' %(len(imgs)/len(links)*100))
        clear_output(wait=True)
    return imgs, steam_links, reviews_counts, positive_reviews, games_names

In [257]:
imgs, steam_links, reviews_counts, positive_reviews, games_names = get_pc_games_info(links)

Processed 100.00% of data


In [259]:
err_links

['https://steamspy.com/app/2240450']

In [260]:
links.index(err_links[0])

68

In [261]:
positive_reviews[68] = 0
reviews_counts[68] = 0

In [262]:
df = pd.DataFrame()

In [265]:
df['Game'] = games_names
df['Image'] = imgs
df['Owners'] = owners
df['Rewiews'] = reviews_counts
df['Positive_Reviews'] = positive_reviews
df['Release_Date'] = release_dates
df['Steam_Link'] = steam_links
df['SteamSpy_Link'] = links
df['Platform'] = 'PC'

In [330]:
df.to_excel(r'F:\test_tasks\RetroStyle_Games\viking_games.xlsx')

In [267]:
google_play = 'https://play.google.com/store/search?q=viking&c=apps&hl=ru&gl=US'

In [355]:
request = requests.get(google_play)
soup = BeautifulSoup(request.content, "html.parser")

In [356]:
games = soup.find_all('a', {'class': 'Si6A0c Gy4nib'})
links = [game.get('href') for game in games]
google_play_startlink = 'https://play.google.com'

In [365]:
err_links = []
def get_mobile_games(links):
    ratings = []
    reviews = []
    downloads = []
    images = []
    names = []
    for link in links:
        link = google_play_startlink+link
        request = requests.get(link)
        soup = BeautifulSoup(request.content, "html.parser")
        try:
            rating = soup.find('div', {'class': 'TT9eCd'}).text
        except:
            rating = '-'
        try:
            review = soup.find('div', {'class': 'g1rdde'}).text
        except:
            review = '-'
        try:
            download = soup.find_all('div', {'class': 'ClM7O'})[1].text
        except:
            download = '-'
        try:
            image = soup.find('div', {'class': 'qxNhq'}).find('img').get('src')
        except:
            image = '-'
        try:
            name = soup.find('h1').find('span').text
        except:
            name = '-'
            
                               
        ratings.append(rating)
        reviews.append(review)
        downloads.append(download)
        images.append(image)
        names.append(name)

        print('Processed %.2f%% of data' %(len(images)/len(links)*100))
        clear_output(wait=True)
    return ratings, reviews, downloads, images, names

In [366]:
ratings, reviews, downloads, images, names = get_mobile_games(links)

Processed 100.00% of data


In [400]:
dff = pd.DataFrame()

In [401]:
dff['Game'] = names
dff['Image'] = images
dff['Owners'] = downloads
dff['Reviews'] = reviews
dff['Positive_Reviews'] = ratings
dff['Platform'] = 'Mobile'

In [402]:
dff = dff[~(dff['Reviews'] == 'Downloads')]

In [403]:
dff.Owners = dff.Owners.str.replace('M\+', '000000')
dff.Owners = dff.Owners.str.replace('K\+', '000')

C:\Users\Nastya\AppData\Local\Temp\ipykernel_16088\4128694429.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dff.Owners = dff.Owners.str.replace('M\+', '000000')
C:\Users\Nastya\AppData\Local\Temp\ipykernel_16088\4128694429.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dff.Owners = dff.Owners.str.replace('K\+', '000')


In [405]:
dff.Reviews = dff.Reviews.str.replace(' reviews', '')

In [407]:
dff.Positive_Reviews = dff.Positive_Reviews.str.replace('star', '')

In [417]:
dff['Stars'] = dff.Positive_Reviews
dff

,Game,Image,Owners,Reviews,Positive_Reviews,Platform,Stars
0,Viking Rise,https://play-lh.googleusercontent.com/_7jYxlA0...,1000000,173K,4.5,Mobile,4.5
1,Vikings: War of Clans,https://play-lh.googleusercontent.com/7vDnSzzv...,10000000,1.01M,4.4,Mobile,4.4
2,Vikingard,https://play-lh.googleusercontent.com/ZSWlD9aM...,1000000,52K,4.4,Mobile,4.4
3,The Last Vikings,https://play-lh.googleusercontent.com/iiKGXLkH...,1000000,110K,4.4,Mobile,4.4
4,Vikings: Valhalla Saga,https://play-lh.googleusercontent.com/nwmYKoOM...,100000,2.39K,3.7,Mobile,3.7
5,Idle Vikings: Viking Tycoon,-,10000,302,4.3,Mobile,4.3
6,"I, Viking: Epic Vikings War fo",https://play-lh.googleusercontent.com/tAntLWAJ...,100000,17.1K,4.2,Mobile,4.2
7,Vikings at War,https://play-lh.googleusercontent.com/qXJblqaV...,1000000,17.1K,4.2,Mobile,4.2
8,Vikings - Age of Warlords,https://play-lh.googleusercontent.com/vdySnm5B...,1000000,32.3K,4.2,Mobile,4.2
9,Viking Voyager,-,100000,1.11K,4.5,Mobile,4.5


In [422]:
dff.Positive_Reviews = dff.Positive_Reviews.astype(float)

In [423]:
dff.Positive_Reviews = dff.Positive_Reviews/5*100

In [424]:
dff

,Game,Image,Owners,Reviews,Positive_Reviews,Platform,Stars
0,Viking Rise,https://play-lh.googleusercontent.com/_7jYxlA0...,1000000,173K,90.0,Mobile,4.5
1,Vikings: War of Clans,https://play-lh.googleusercontent.com/7vDnSzzv...,10000000,1.01M,88.0,Mobile,4.4
2,Vikingard,https://play-lh.googleusercontent.com/ZSWlD9aM...,1000000,52K,88.0,Mobile,4.4
3,The Last Vikings,https://play-lh.googleusercontent.com/iiKGXLkH...,1000000,110K,88.0,Mobile,4.4
4,Vikings: Valhalla Saga,https://play-lh.googleusercontent.com/nwmYKoOM...,100000,2.39K,74.0,Mobile,3.7
5,Idle Vikings: Viking Tycoon,-,10000,302,86.0,Mobile,4.3
6,"I, Viking: Epic Vikings War fo",https://play-lh.googleusercontent.com/tAntLWAJ...,100000,17.1K,84.0,Mobile,4.2
7,Vikings at War,https://play-lh.googleusercontent.com/qXJblqaV...,1000000,17.1K,84.0,Mobile,4.2
8,Vikings - Age of Warlords,https://play-lh.googleusercontent.com/vdySnm5B...,1000000,32.3K,84.0,Mobile,4.2
9,Viking Voyager,-,100000,1.11K,90.0,Mobile,4.5


In [411]:
df

,Game,Image,Owners,Rewiews,Positive_Reviews,Release_Date,Steam_Link,SteamSpy_Link,Platform
0,Northgard,https://steamcdn-a.akamaihd.net/steam/apps/466...,"2,000,000 .. 5,000,000",42157,88,2018-03-07,http://store.steampowered.com/app/466560,https://steamspy.com/app/466560,PC
1,Jotun: Valhalla Edition,https://steamcdn-a.akamaihd.net/steam/apps/323...,"2,000,000 .. 5,000,000",1969,78,2015-09-29,http://store.steampowered.com/app/323580,https://steamspy.com/app/323580,PC
2,"Pirates, Vikings, and Knights II",https://steamcdn-a.akamaihd.net/steam/apps/175...,"1,000,000 .. 2,000,000",5634,87,2010-02-12,http://store.steampowered.com/app/17570,https://steamspy.com/app/17570,PC
3,Tribes of Midgard,https://steamcdn-a.akamaihd.net/steam/apps/858...,"1,000,000 .. 2,000,000",13115,75,2021-07-27,http://store.steampowered.com/app/858820,https://steamspy.com/app/858820,PC
4,The Banner Saga,https://steamcdn-a.akamaihd.net/steam/apps/237...,"500,000 .. 1,000,000",12326,88,2014-01-14,http://store.steampowered.com/app/237990,https://steamspy.com/app/237990,PC
...,...,...,...,...,...,...,...,...,...
66,Viking Brothers 4,https://steamcdn-a.akamaihd.net/steam/apps/836...,"0 .. 20,000",20,95,2018-04-25,http://store.steampowered.com/app/836560,https://steamspy.com/app/836560,PC
67,Viking Brothers 2,https://steamcdn-a.akamaihd.net/steam/apps/836...,"0 .. 20,000",26,88,2018-04-30,http://store.steampowered.com/app/836540,https://steamspy.com/app/836540,PC
68,Helga the Viking Warrior 2: Ivar's Revenge,https://steamcdn-a.akamaihd.net/steam/apps/224...,"0 .. 20,000",0,0,2022-12-29,http://store.steampowered.com/app/2240450,https://steamspy.com/app/2240450,PC
69,Prophecy I - The Viking Child,https://steamcdn-a.akamaihd.net/steam/apps/719...,"0 .. 20,000",5,Need more user reviews to generate a score,1991-07-01,http://store.steampowered.com/app/719290,https://steamspy.com/app/719290,PC


In [414]:
df.Owners = df.Owners.str.replace(',', '')
df[['OwnersMin', 'OwnersMax']] = df.Owners.str.split(' .. ', expand=True).astype(int)
df['OwnersMean'] = df['OwnersMin']+df['OwnersMax']//2

In [415]:
df

,Game,Image,Owners,Rewiews,Positive_Reviews,Release_Date,Steam_Link,SteamSpy_Link,Platform,OwnersMin,OwnersMax,OwnersMean
0,Northgard,https://steamcdn-a.akamaihd.net/steam/apps/466...,2000000 .. 5000000,42157,88,2018-03-07,http://store.steampowered.com/app/466560,https://steamspy.com/app/466560,PC,2000000,5000000,4500000
1,Jotun: Valhalla Edition,https://steamcdn-a.akamaihd.net/steam/apps/323...,2000000 .. 5000000,1969,78,2015-09-29,http://store.steampowered.com/app/323580,https://steamspy.com/app/323580,PC,2000000,5000000,4500000
2,"Pirates, Vikings, and Knights II",https://steamcdn-a.akamaihd.net/steam/apps/175...,1000000 .. 2000000,5634,87,2010-02-12,http://store.steampowered.com/app/17570,https://steamspy.com/app/17570,PC,1000000,2000000,2000000
3,Tribes of Midgard,https://steamcdn-a.akamaihd.net/steam/apps/858...,1000000 .. 2000000,13115,75,2021-07-27,http://store.steampowered.com/app/858820,https://steamspy.com/app/858820,PC,1000000,2000000,2000000
4,The Banner Saga,https://steamcdn-a.akamaihd.net/steam/apps/237...,500000 .. 1000000,12326,88,2014-01-14,http://store.steampowered.com/app/237990,https://steamspy.com/app/237990,PC,500000,1000000,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Viking Brothers 4,https://steamcdn-a.akamaihd.net/steam/apps/836...,0 .. 20000,20,95,2018-04-25,http://store.steampowered.com/app/836560,https://steamspy.com/app/836560,PC,0,20000,10000
67,Viking Brothers 2,https://steamcdn-a.akamaihd.net/steam/apps/836...,0 .. 20000,26,88,2018-04-30,http://store.steampowered.com/app/836540,https://steamspy.com/app/836540,PC,0,20000,10000
68,Helga the Viking Warrior 2: Ivar's Revenge,https://steamcdn-a.akamaihd.net/steam/apps/224...,0 .. 20000,0,0,2022-12-29,http://store.steampowered.com/app/2240450,https://steamspy.com/app/2240450,PC,0,20000,10000
69,Prophecy I - The Viking Child,https://steamcdn-a.akamaihd.net/steam/apps/719...,0 .. 20000,5,Need more user reviews to generate a score,1991-07-01,http://store.steampowered.com/app/719290,https://steamspy.com/app/719290,PC,0,20000,10000


In [430]:
df = df.drop(columns=['Owners', 'OwnersMin', 'OwnersMax'])
df = df.rename(columns={'OwnersMean': 'Owners', 'Rewiews': 'Reviews'})

In [445]:
df_final = pd.concat([df, dff], ignore_index=True)

In [433]:
df_final.to_excel(r'...\viking_games_final.xlsx')

In [447]:
aa = pd.read_excel(r'...\viking_games_final.xlsx', index_col=0)

In [438]:
aa.Reviews = aa.Reviews.astype(int)

In [440]:
aa.Positive_Reviews = aa.Positive_Reviews.astype(int)

In [442]:
aa.Owners = aa.Owners.astype(int)

In [457]:
aa.to_excel(r'...\viking_games_final1.xlsx')

In [446]:
df_final

,Game,Image,Reviews,Positive_Reviews,Release_Date,Steam_Link,SteamSpy_Link,Platform,Owners,Stars
0,Northgard,https://steamcdn-a.akamaihd.net/steam/apps/466...,42157,88,2018-03-07,http://store.steampowered.com/app/466560,https://steamspy.com/app/466560,PC,4500000,NaN
1,Jotun: Valhalla Edition,https://steamcdn-a.akamaihd.net/steam/apps/323...,1969,78,2015-09-29,http://store.steampowered.com/app/323580,https://steamspy.com/app/323580,PC,4500000,NaN
2,"Pirates, Vikings, and Knights II",https://steamcdn-a.akamaihd.net/steam/apps/175...,5634,87,2010-02-12,http://store.steampowered.com/app/17570,https://steamspy.com/app/17570,PC,2000000,NaN
3,Tribes of Midgard,https://steamcdn-a.akamaihd.net/steam/apps/858...,13115,75,2021-07-27,http://store.steampowered.com/app/858820,https://steamspy.com/app/858820,PC,2000000,NaN
4,The Banner Saga,https://steamcdn-a.akamaihd.net/steam/apps/237...,12326,88,2014-01-14,http://store.steampowered.com/app/237990,https://steamspy.com/app/237990,PC,1000000,NaN
...,...,...,...,...,...,...,...,...,...,...
94,Viking Saga 2: Northern World,https://play-lh.googleusercontent.com/ePYPYyGx...,2.37K,82.0,NaN,NaN,NaN,Mobile,100000,4.1
95,Vikings II,https://play-lh.googleusercontent.com/JW8CvFXu...,1.86K,68.0,NaN,NaN,NaN,Mobile,100000,3.4
96,My Viking,-,3.86K,84.0,NaN,NaN,NaN,Mobile,100000,4.2
97,Vikings and Dragon Island Farm,https://play-lh.googleusercontent.com/YK3CB7F1...,14.9K,90.0,NaN,NaN,NaN,Mobile,1000000,4.5


In [448]:
aa

,Game,Image,Reviews,Positive_Reviews,Release_Date,Platform,Owners
72,Vikings: War of Clans,https://play-lh.googleusercontent.com/7vDnSzzv...,1010000,88,2015-08-10,Mobile,10000000
0,Northgard,https://steamcdn-a.akamaihd.net/steam/apps/466...,42157,88,2018-03-07,PC,4500000
1,Jotun: Valhalla Edition,https://steamcdn-a.akamaihd.net/steam/apps/323...,1969,78,2015-09-29,PC,4500000
2,"Pirates, Vikings, and Knights II",https://steamcdn-a.akamaihd.net/steam/apps/175...,5634,87,2010-02-12,PC,2000000
3,Tribes of Midgard,https://steamcdn-a.akamaihd.net/steam/apps/858...,13115,75,2021-07-27,PC,2000000
...,...,...,...,...,...,...,...
66,Viking Brothers 4,https://steamcdn-a.akamaihd.net/steam/apps/836...,20,95,2018-04-25,PC,10000
67,Viking Brothers 2,https://steamcdn-a.akamaihd.net/steam/apps/836...,26,88,2018-04-30,PC,10000
76,Idle Vikings: Viking Tycoon,https://play-lh.googleusercontent.com/3qv-Qu9y...,302,86,2020-02-18,Mobile,10000
83,Viking Kingdom: Ragnarok Age,https://play-lh.googleusercontent.com/3w3kzTkT...,165,60,2022-08-31,Mobile,10000


In [449]:
aa['Release_Date'] = pd.to_datetime(aa['Release_Date'], format='%Y-%m-%d')
aa['Days_Since_Release'] = (dt.now() - aa.Release_Date).dt.days

In [451]:
aa = aa.drop(columns='Release_Date')

In [453]:
aa['Reviews_Per_Day'] = aa['Reviews']//aa['Days_Since_Release']